Julia language

## Exercise 1 ( 10 min )

(a) write a function that constructs a quantum `state` in z-basis for N qbit system in vector format.

Input : string of bits "..000101101.."

(b) write a function that constructs a quantum `Hamiltonian` for N qbit system in matrix format

$H = \sum n_i n_{i+1} - \mu\sum n_i$

where $n = \begin{bmatrix}
           1 & 0 \\
           0 & 0 \\
         \end{bmatrix}$
         
Input: number of qbits N and chemical potential $\mu$ 


hint: one can apply kronecker product iteratively with `foldl(⊗, container)`


In [1]:
⊗(x, y) = kron(x, y)


function psi(bitstring)
    up = [1, 0]
    down = [0, 1]
    state = fill(up, length(bitstring))
    for (i, char) in enumerate(bitstring)
        if char == '0'
            state[i] = down
        end
    end 
    return foldl(⊗, state) #reduce 
end



function Ham(N, μ)  # μ is \mu<TAB>
    id = [
        1 0;0 1]
    ProjUp = [1 0; 0 0]
    H = zeros(2^N, 2^N)
    
    for i in 1: N
        one_body = fill(id, N)
        one_body[i] = ProjUp 
        H += -μ * foldl(⊗, one_body)
    end
    
    for i in 1: N-1
        two_body = fill(id, N)
        two_body[i] = ProjUp
        two_body[i+1] = ProjUp
        H += foldl(⊗, two_body)
    end
    return H
end



Ham (generic function with 1 method)

In [2]:
psi("11")
Ham(2, 0.1)

4×4 Matrix{Float64}:
 0.8   0.0   0.0  0.0
 0.0  -0.1   0.0  0.0
 0.0   0.0  -0.1  0.0
 0.0   0.0   0.0  0.0

## Exercise 2 ( 10 min )

write a function that returns a product state(s) with minimal energy, i.e. 

$|\psi\rangle : \langle \psi |H(N, \mu)| \psi \rangle = E_{min}$

among z-basis states ("..000101101..").

$H = \sum n_i n_{i+1} - \mu\sum n_i$


answer should have a following format `Pair("..0100100..", psi)`

hint: function that translates integer to bitsting is `string(i; base = 2, pad = string_length)`


In [3]:
function psi_min(H)
    s=size(H, 1) #size along dim 1
    N = Int(log2(s))
    
    #Number = {Int, Int32, Int64, Float, Float32, Float64... }
    
    energy = Dict{String, Number}()
    for i in 0 : 2^N-1 
        bstr = string(i; base = 2, pad = N)
        ψ = psi(bstr)
        energy[bstr] = (ψ' * H * ψ)
    end
    min_en = findmin(energy)[1] #finding a min
    return filter(elem->(elem.second == min_en), energy) #filter out enrgies ≠ min
end

psi_min (generic function with 1 method)

In [4]:
@time psi_min(Ham(10, 0.4))

  0.298671 seconds (96.02 k allocations: 479.650 MiB, 4.96% gc time, 7.43% compilation time)


Dict{String, Number} with 6 entries:
  "1010100101" => -2.0
  "1001010101" => -2.0
  "0101010101" => -2.0
  "1010101010" => -2.0
  "1010101001" => -2.0
  "1010010101" => -2.0

## Exercise 3 ( 5 min )

Write `time_evolution` function, that evolves a state in time from 0 to `T` with time step `dt`. 
Evolution operator is $e^{-iHt}$


hint: its much cheaper to compute $e^{At} * v$ then $e^{At}$ and apply it to $v$.

hint: one can test `time_evolution` implementation with complex random normalized vector of size `2^N`

Use `exponentiate` by `using KrylovKit` package, which returns updated vector and convergence info `v, info`

In [5]:
using KrylovKit: exponentiate

function time_evolution(ψ, H, T, dt)
    n = Int(T/dt)
    for _ in 1:n #pay attention!
    #in Julia ranges are closed, i.e 1:5 means 1,2,3,4,5
        ψ, info = exponentiate(H, -dt * 1.0im, ψ)
    end
    return ψ
end


time_evolution (generic function with 1 method)

In [10]:
# Usage example

using LinearAlgebra

L = 5
ψ = rand(Complex{Float64}, 2^L)
ψ /= norm(ψ,2)

time_evolution(ψ, Ham(L,0.1), 5.0, 0.1)

32-element Vector{ComplexF64}:
  -0.11369655122605653 + 0.0644759408528872im
   0.18349136743927763 - 0.018804816011487188im
    0.1467577801718119 - 0.028354675851863194im
  0.026688228912223316 - 0.1124675819945794im
   0.15798822035534396 - 0.15212888374558808im
  -0.21275316455822024 - 0.023731124236512875im
  -0.12136589943180677 - 0.06088868446034836im
  -0.13852381519551052 - 0.0691409971171647im
   0.01834366369626367 - 0.11974752725919376im
  -0.07862415197345206 - 0.0635506699095704im
  -0.14791132144988767 + 0.19550749202417048im
  -0.08707132251266379 + 0.2099442167096762im
  -0.20052000333458775 - 0.06848655240761178im
                       ⋮
  -0.18249832705793287 - 0.10218628862041482im
   0.06809470175546742 + 0.1264850558212589im
   -0.0620675827036714 + 0.1621622664192347im
 -0.020759338229685908 + 0.07421231764004241im
   0.10253211481695645 - 0.10792728617841092im
  -0.19711912650548005 - 0.07048267356892052im
  0.030117474287823014 + 0.12083556505161931im
   0.050

## Exercise 4 ( 10 min )

compare execution time of `time_evolution` function (Ex 3) for different types of Hamiltonians `H(N = 14, μ = 0.5)`

1. dense
2. sparse `using SparseArrays`
3. OPTIONAL : cuda dense (for GPU)

hint: `CUDA` package has a `cu()` function that translates arrays to GPU.
    

execution time can be measured with macro @time (or better @btime from `BenchmarkTools`)

In [18]:
using SparseArrays
using BenchmarkTools
using LinearAlgebra


L = 12
dt = 0.1
T = 5.0
H = Ham(L, 0.1)
Hsp = sparse(H)

ψ = rand(Complex{Float64}, 2^L)
ψ /= norm(ψ,2)

@time time_evolution(ψ, H, T, dt)
@time time_evolution(ψ, Hsp, T, dt)


 11.944268 seconds (4.55 k allocations: 118.975 MiB)
  1.511705 seconds (4.65 k allocations: 120.543 MiB)


4096-element Vector{ComplexF64}:
  -0.011853896873262104 + 0.01259998223099739im
   0.016679847637844233 + 0.004370082560263621im
   0.002114974256463454 - 0.0071131515450265324im
 -0.0032054591819539137 - 0.00926459570257121im
  0.0011641368741503472 - 0.005278561585997654im
  -0.009475273607725068 - 0.005131796471141815im
  -0.022962912845868162 - 0.00540743868054884im
  -0.006543091664889295 + 0.0008557358944384494im
   0.004189808193965449 - 0.005722135568732727im
  -0.014262943252438908 - 0.013317884178562809im
  -0.006759735280905994 + 0.006290766876914184im
 -0.0023149785963365744 + 0.024138828713581504im
   -0.01298834674972344 - 0.0033500004175022738im
                        ⋮
   -0.01402220755017167 - 0.012596739847918991im
   -0.00611078532243405 + 0.021092781754763045im
   0.005161613712920969 + 0.010110514799360763im
  -0.004415839482049863 + 0.013229135261676957im
  0.0015452517633196218 - 0.002950560010065211im
  -0.003299943671265516 + 0.0011637286724671902im
 0.000109

In [14]:
using SparseArrays
using BenchmarkTools
using LinearAlgebra

L = 10
dt = 0.1
T = 5.0
H = Ham(L, 0.1)
Hsp = sparse(H)

1024×1024 SparseMatrixCSC{Float64, Int64} with 1023 stored entries:
⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄